In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
train=pd.read_csv("/kaggle/input/ventilator-pressure-prediction/train.csv")
test=pd.read_csv("/kaggle/input/ventilator-pressure-prediction/test.csv")
train

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
f, ax=plt.subplots(1,3, figsize=(20,5))
sns.countplot(x='R', data=train, saturation=1, dodge=True, ax=ax[0])
ax[0].set_title("Change in pressure per change in flow (air volume per time)", fontsize=10)
sns.countplot(x='C', data=train, saturation=0.2, dodge=True, ax=ax[1])
ax[1].set_title("Change in volume per change in pressure", fontsize=10)
sns.countplot(x='u_out', data=train, saturation=2, dodge=True, ax=ax[2])
ax[2].set_title("Control input for the exploratory solenoid valve", fontsize=10)

In [ ]:
f, ax=plt.subplots(1,3, figsize=(20,5))
sns.boxplot(x='time_step', data=train, saturation=1, dodge=True, ax=ax[0])
ax[0].set_title("time_step", fontsize=12)
sns.boxplot(x='u_in', data=train, saturation=0.2, dodge=True, ax=ax[1])
ax[1].set_title("u_in", fontsize=12)
sns.boxplot(x='pressure', data=train, saturation=2, dodge=True, ax=ax[2])
ax[2].set_title("pressure", fontsize=14)

In [ ]:
f, ax=plt.subplots(1,3, figsize=(20,5))
sns.violinplot(x='time_step', data=train, saturation=1, dodge=True, ax=ax[0], color='g')
ax[0].set_title("time_step", fontsize=12)
sns.violinplot(x='u_in', data=train, saturation=0.2, dodge=True, ax=ax[1], color='g')
ax[1].set_title("u_in", fontsize=12)
sns.violinplot(x='pressure', data=train, saturation=2, dodge=True, ax=ax[2], color='g')
ax[2].set_title("pressure", fontsize=14)

In [ ]:
f, ax=plt.subplots(1,3, figsize=(20,5))
sns.distplot(x=train['time_step'], ax=ax[0])
ax[0].set_title("time_step", fontsize=12)
sns.distplot(train['u_in'],  ax=ax[1])
ax[1].set_title("u_in", fontsize=12)
sns.distplot(train['pressure'], ax=ax[2])
ax[2].set_title("pressure", fontsize=14)

# Categorical variables in test data

In [ ]:
f, ax=plt.subplots(1,3, figsize=(20,5))
sns.countplot(x='R', data=test, saturation=1, dodge=True, ax=ax[0])
ax[0].set_title("Change in pressure per change in flow (air volume per time)", fontsize=10)
sns.countplot(x='C', data=test, saturation=0.2, dodge=True, ax=ax[1])
ax[1].set_title("Change in volume per change in pressure", fontsize=10)
sns.countplot(x='u_out', data=test, saturation=2, dodge=True, ax=ax[2])
ax[2].set_title("Control input for the exploratory solenoid valve", fontsize=10)

# Outliers in test data¶

In [ ]:
f, ax=plt.subplots(1,2, figsize=(20,5))
sns.boxplot(x='time_step', data=train, saturation=1, dodge=True, ax=ax[0])
ax[0].set_title("time_step", fontsize=12)
sns.boxplot(x='u_in', data=train, saturation=0.2, dodge=True, ax=ax[1])
ax[1].set_title("u_in", fontsize=12)

In [ ]:
f, ax=plt.subplots(1,2, figsize=(20,5))
sns.distplot(x=test['time_step'], ax=ax[0])
ax[0].set_title("time_step", fontsize=12)
sns.distplot(test['u_in'],  ax=ax[1])
ax[1].set_title("u_in", fontsize=12)

# Skewness and Kurtosis

In [ ]:
print("Skewness for time_out in train data", str(round(train['time_step'].skew(axis=0, skipna=True),2)))
print("Skewness for time_out train data", str(round(train['time_step'].kurtosis(axis=0, skipna=True),2)))
print("--------------------------")
print("Skewness for u_in train data", str(round(train['u_in'].skew(axis=0, skipna=True),2)))
print("Skewness for u_in train data", str(round(train['u_in'].kurtosis(axis=0, skipna=True),2)))
print("--------------------------")
print("Skewness for pressure train data", str(round(train['pressure'].skew(axis=0, skipna=True),2)))
print("Skewness for pressure train data", str(round(train['pressure'].kurtosis(axis=0, skipna=True),2)))

In [ ]:
print("Skewness for time_out in test data", str(round(test['time_step'].skew(axis=0, skipna=True),2)))
print("Skewness for time_out test data", str(round(test['time_step'].kurtosis(axis=0, skipna=True),2)))
print("--------------------------")
print("Skewness for u_in test data", str(round(test['u_in'].skew(axis=0, skipna=True),2)))
print("Skewness for u_in test data", str(round(test['u_in'].kurtosis(axis=0, skipna=True),2)))

# Outlier removal


In [ ]:
q25= train.quantile(0.25)
q75 = train.quantile(0.75)
IQR= q75 - q25
print(IQR)

In [ ]:
train_filter = train[((train < (IQR - 3*q25)) | (train > (IQR + 3*q75))).any(axis=1)]
train_filter

In [ ]:
f, ax=plt.subplots(1,3, figsize=(20,5))
sns.boxplot(x='time_step', data=train_filter, saturation=1, dodge=True, ax=ax[0])
ax[0].set_title("time_step", fontsize=12)
sns.boxplot(x='u_in', data=train_filter, saturation=0.2, dodge=True, ax=ax[1])
ax[1].set_title("u_in", fontsize=12)
sns.boxplot(x='pressure', data=train_filter, saturation=2, dodge=True, ax=ax[2])
ax[2].set_title("pressure", fontsize=14)

In [ ]:
print("Skewness for time_out in filtered train data", str(round(train_filter['time_step'].skew(axis=0, skipna=True),2)))
print("Skewness for time_out filtered train data", str(round(train_filter['time_step'].kurtosis(axis=0, skipna=True),2)))
print("--------------------------")
print("Skewness for u_in filtered train data", str(round(train_filter['u_in'].skew(axis=0, skipna=True),2)))
print("Skewness for u_in filtered train data", str(round(train_filter['u_in'].kurtosis(axis=0, skipna=True),2)))
print("--------------------------")
print("Skewness for pressure filtered train data", str(round(train_filter['pressure'].skew(axis=0, skipna=True),2)))
print("Skewness for pressure filtered train data", str(round(train_filter['pressure'].kurtosis(axis=0, skipna=True),2)))

# Train Test Splits

In [ ]:
train_filter.drop('id', axis=1, inplace=True)
train_filter

In [ ]:
from sklearn.model_selection import train_test_split
X=train_filter.drop('pressure', axis=1)
y=train_filter['pressure']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)

In [ ]:
from catboost import CatBoostRegressor

from sklearn.model_selection import cross_val_score

In [ ]:
catboost=CatBoostRegressor()
catboost_train_score=cross_val_score(catboost, X_train, y_train, cv=5)

In [ ]:
print(catboost_train_score.mean())

# Hypertuning

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
catboost=CatBoostRegressor()
grid={'depth': [6,8,10],
      'learning_rate' : [0.01, 0.05, 0.1],
      'iterations'    : [30, 50, 100]}

In [ ]:
grid_catboost=GridSearchCV(estimator=catboost, param_grid = grid, cv = 2, n_jobs=-1)

In [ ]:
grid_catboost.fit(X_train,y_train)

In [ ]:
print(" Results from Grid Search " )
print("Best estimator",grid_catboost.best_estimator_)
print("Best parameters" ,grid_catboost.best_params_)
print("Best score", grid_catboost.best_score_)

In [ ]:
from lightgbm import LGBMRegressor

In [ ]:
lgbm=LGBMRegressor()
lgbm.fit(X_train, y_train)

In [ ]:
train_score = lgbm.score(X_train, y_train)
print(train_score)

In [ ]:
test_score = lgbm.score(X_test, y_test)
print(test_score)

In [ ]:
from xgboost import XGBRegressor

In [ ]:
xgb = XGBRegressor()
xgb.fit(X_train, y_train)
tr_sc=xgb.score(X_train, y_train)

In [ ]:
te_sc=xgb.score(X_test, y_test)
print(te_sc)